In [ ]:
import pandas as pd

from constants import CFB26_ATTRS, CROP_LOCATIONS, GOING_RIGHT_PATHS


pd.set_option('display.max_columns', None)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np




def show_image_plt(image:np.array):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=(20, 16))
    ax.imshow(image_rgb)
    plt.axis('off')
    plt.show()


def show_image_with_box(image_path, x, y, w, h, alpha=0.15):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=(10*3, 8*3))
    ax.imshow(image_rgb)

    # Transparent rectangle
    rect = patches.Rectangle((x, y), w, h,
                             linewidth=2,
                             edgecolor='yellow',
                             facecolor='yellow',
                             alpha=alpha)
    ax.add_patch(rect)
    ax.set_title(f"Box: (x={x}, y={y}, w={w}, h={h})")
    plt.axis('off')
    plt.show()



# path = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250802_171000_000049.jpg"
path = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250804_004207_000000.jpg"
# rect_locs = CROP_LOCATIONS['SCORES_SCHEDULE']['table_data']
CROP_LOCATIONS = {
    'SCORES_SCHEDULE': {
        'page_title': {'x': 40, 'y': 20, 'w': 500, 'h': 40},
        'table_data': {'x': 60, 'y': 265, 'w': 860, 'h': 349},
    }
}
rect_locs = CROP_LOCATIONS['SCORES_SCHEDULE']['page_title']
show_image_with_box(path, **rect_locs)



In [ ]:

def show_image_with_box(image_path, crop_locations, alpha=0.15):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=(10*3, 8*3))
    ax.imshow(image_rgb)

    for label, coords in crop_locations.items(): #CROP_LOCATIONS['SCORES_SCHEDULE'].items():
        x, y, w, h = coords['x'], coords['y'], coords['w'], coords['h']
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=d_colors[label], facecolor='none', label=label)
        ax.add_patch(rect)
        ax.text(x, y - 5, label, color=d_colors[label], fontsize=9)

    # Transparent rectangle
    # rect = patches.Rectangle((x, y), w, h,
    #                          linewidth=2,
    #                          edgecolor='yellow',
    #                          facecolor='yellow',
    #                          alpha=alpha)
    # ax.add_patch(rect)
    # ax.set_title(f"Box: (x={x}, y={y}, w={w}, h={h})")
    plt.axis('off')
    plt.show()
    


d_colors = {
    'page_title': 'lightblue',
    'table_data': 'green',
    'left_trigger': 'orange',
    'left_trigger': 'orange',
    'right_trigger': 'pink'
}
CROP_LOCATIONS = {
    'SCORES_SCHEDULE': {
        'page_title': {'x': 40, 'y': 20, 'w': 500, 'h': 40},
        'table_data': {'x': 60, 'y': 200, 'w': 860, 'h': 414},
        'left_trigger': {'x': 93, 'y': 126, 'w': 210, 'h': 24},
        'right_trigger': {'x': 675, 'y': 126, 'w': 210, 'h': 24},
    }
}

    
show_image_with_box(path, CROP_LOCATIONS['SCORES_SCHEDULE'])


# fig, ax = plt.subplots(figsize=(10, 6))


# # Loop over each box in SCORES_SCHEDULE
# for label, coords in boxes['SCORES_SCHEDULE'].items():
#     x, y, w, h = coords['x'], coords['y'], coords['w'], coords['h']
#     rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=d_colors[label], facecolor='none', label=label)
#     ax.add_patch(rect)
#     ax.text(x, y - 5, label, color=d_colors[label], fontsize=9)

# # Adjust axes
# ax.set_xlim(0, 1000)
# ax.set_ylim(0, 700)
# ax.invert_yaxis()  # Invert if origin is top-left like in image coordinates
# plt.title("Box Annotations from SCORES_SCHEDULE")
# plt.grid(True)
# plt.legend()
# plt.show()

In [ ]:
import numpy as np

import easyocr


def crop_region(image_input, x, y, w, h):
    if isinstance(image_input, str):
        image = cv2.imread(image_input)
    elif isinstance(image_input, np.ndarray):
        image = image_input
    else:
        raise TypeError("image_input must be a filepath (str) or a numpy.ndarray")
    
    cropped = image[y:y+h, x:x+w]
    return cropped

def preprocess_image(image_input):
    if isinstance(image_input, str):
        image = cv2.imread(image_input)
    elif isinstance(image_input, np.ndarray):
        image = image_input
    else:
        raise TypeError("image_input must be a filepath (str) or a numpy.ndarray")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Optional: increase contrast or sharpen
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.adaptiveThreshold(blur, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY_INV, 11, 2)
    return thresh

def enhance_contrast(image):
    """Apply CLAHE to enhance local contrast."""
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    merged = cv2.merge((cl, a, b))
    result = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    return result

def denoise_image(image):
    """Apply bilateral filter for edge-preserving denoising."""
    return cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

def run_easyocr_on_image(image_path, use_gpu=True, preprocess=False, show_image=False, denoise=False, enhance_contrast=False, crop_loc=None):
    image_input = image_path
    reader = easyocr.Reader(['en'], gpu=use_gpu)

    if preprocess:
        image_input = preprocess_image(image_input)
    if crop_loc is not None:
        image_input = crop_region(image_input, **crop_loc)

    if denoise:
        image_input = denoise_image(image_input)
    if enhance_contrast:
        image_input = enhance_contrast(image_input)
    image_input = cv2.resize(image_input, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    if show_image == True:
        show_image_plt(image_input)

    results = reader.readtext(image_input)
    # for bbox, text, conf in results:
    #     print(f"{text} ({conf:.2f}) | {bbox}")
    return results


path = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250802_171000_000049.jpg"

data = run_easyocr_on_image(
    image_path = path, 
    use_gpu = False, 
    preprocess = False, 
    denoise = False,
    enhance_contrast = False,
    show_image = False,
    crop_loc = CROP_LOCATIONS['ROSTER']['table_data']
)
print(data)

In [ ]:
# import json
# import numpy as np
# import pandas as pd
# from collections import defaultdict

# def vertical_center(bbox):
#     return sum(p[1] for p in bbox) / 4


# def group_ocr_rows(data, y_tolerance=10):
#     rows = defaultdict(list)
#     for bbox, text, conf in data:
#         y_center = vertical_center(bbox)
#         found = False
#         for key in rows:
#             if abs(key - y_center) < y_tolerance:
#                 rows[key].append((bbox, text))
#                 found = True
#                 break
#         if not found:
#             rows[y_center].append((bbox, text))
#     # Sort rows vertically
#     sorted_rows = [rows[k] for k in sorted(rows)]
#     return sorted_rows



# path = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250802_171000_000049.jpg"
# data = run_easyocr_on_image(
#     image_path = path, 
#     use_gpu = False, 
#     preprocess = False, 
#     show_image = True,
#     crop_loc = roster_data_loc
# )

# headers = [d[1] for d in data if d[0][0][1] <= 30]
# body_data = [d for d in data if d[0][0][1] > 30]
# grouped_rows = group_ocr_rows(body_data)
# print(grouped_rows[:2])
# rows = [row[1] for row in grouped_rows[0]]
# print(json.dumps(rows, indent=4))

In [ ]:
# def build_table(headers, grouped_rows):
#     table = []
#     for row in grouped_rows:
#         # Sort left to right by bbox X-coordinate
#         sorted_row = sorted(row, key=lambda x: x[0][0][0])
#         row_text = [t[1] for t in sorted_row]

#         if len(row_text) == len(headers):
#             table.append(row_text)
#         else:
#             table.append([np.nan] * len(headers))
#     return pd.DataFrame(table, columns=headers)

# grouped_rows = group_ocr_rows(body_data)
# df_49 = build_table(headers, grouped_rows)
# print(df_49)

In [ ]:


grouped_rows = group_ocr_rows(body_data)
df = build_table(headers, grouped_rows)
print(df)

In [ ]:
file = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250802_170956_000033.jpg"

from pathlib import Path

folder = Path("/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/")

paths = []
for file in folder.iterdir():
    if file.is_file():
        path = str(file)
        if path[-7:] != "log.csv":
            # print(file.resolve())
            paths.append(path)
paths = sorted(paths)
print(f"paths(len={len(paths)}) = {paths}")

In [ ]:
numeric_cols = ['OVR', 'SPD', 'ACC', 'AGI', 'COD', 'STR', 'AWR', 'THP', 'SAC', 'MAC']

import json
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.spatial.distance import euclidean




def vertical_center(bbox):
    return sum(p[1] for p in bbox) / 4


def group_ocr_rows(data, y_tolerance=10):
    rows = defaultdict(list)
    for bbox, text, conf in data:
        y_center = vertical_center(bbox)
        found = False
        for key in rows:
            if abs(key - y_center) < y_tolerance:
                rows[key].append((bbox, text))
                found = True
                break
        if not found:
            rows[y_center].append((bbox, text))
    # Sort rows vertically
    sorted_rows = [rows[k] for k in sorted(rows)]
    return sorted_rows



def build_table(headers, grouped_rows):
    table = []
    for row in grouped_rows:
        # Sort left to right by bbox X-coordinate
        sorted_row = sorted(row, key=lambda x: x[0][0][0])
        row_text = [t[1] for t in sorted_row]

        if len(row_text) == len(headers):
            table.append(row_text)
        else:
            table.append([np.nan] * len(headers))
    return pd.DataFrame(table, columns=headers)




def row_fingerprint(row, cols):
    fp = []
    for col in cols:
        val = row.get(col)
        if pd.notna(val):
            try:
                fp.append(float(val))
            except ValueError:
                continue  # skip if value is not numeric
    return tuple(fp)


def fingerprint_similarity(f1, f2):
    if len(f1) != len(f2):
        return float('inf')  # avoid comparing rows with different lengths
    return euclidean(f1, f2)

def align_frames(df_prev, df_curr, numeric_cols, max_distance=5.0):
    matched_rows = []
    used_prev_indices = set()

    for i, curr_row in df_curr.iterrows():
        if curr_row.isnull().all():
            matched_rows.append((None, i))  # missing row
            continue

        curr_fp = row_fingerprint(curr_row, numeric_cols)
        best_match = None
        best_score = float('inf')
        best_j = None

        for j, prev_row in df_prev.iterrows():
            if j in used_prev_indices or prev_row.isnull().all():
                continue
            prev_fp = row_fingerprint(prev_row, numeric_cols)
            score = fingerprint_similarity(curr_fp, prev_fp)

            if score < best_score and score <= max_distance:
                best_score = score
                best_j = j
                best_match = prev_row

        if best_j is not None:
            used_prev_indices.add(best_j)
            matched_rows.append((best_j, i))
        else:
            matched_rows.append((None, i))

    return matched_rows

def merge_frames(df_prev, df_curr, numeric_cols):
    alignment = align_frames(df_prev, df_curr, numeric_cols)
    print(f"len(alignment) = {len(alignment)}")
    merged = []

    used_prev_indices = set()
    used_curr_indices = set()

    # First: merge aligned rows
    for prev_idx, curr_idx in alignment:
        if prev_idx is not None:
            used_prev_indices.add(prev_idx)
        if curr_idx is not None:
            used_curr_indices.add(curr_idx)

        if prev_idx is None:
            merged.append(df_curr.iloc[curr_idx])
        else:
            row = df_curr.iloc[curr_idx].combine_first(df_prev.iloc[prev_idx])
            merged.append(row)

    # Second: append any unmatched rows from df_prev
    for idx in df_prev.index:
        if idx not in used_prev_indices:
            merged.append(df_prev.iloc[idx])

    return pd.DataFrame(merged, columns=df_curr.columns)

def merge_frames_list(df_list, numeric_cols, max_distance=5.0, match_window=10):
    merged_rows = []

    for i, df in enumerate(df_list):
        if i == 0:
            merged_rows.extend(df.to_dict('records'))
            continue

        for _, row in df.iterrows():
            if row.isnull().all():
                continue

            row_fp = row_fingerprint(row, numeric_cols)
            found_match = False

            # Only search in last N rows
            search_start = max(0, len(merged_rows) - match_window)
            for existing_row in merged_rows[search_start:]:
                existing_fp = row_fingerprint(existing_row, numeric_cols)
                score = fingerprint_similarity(row_fp, existing_fp)

                if score <= max_distance:
                    found_match = True
                    break

            if not found_match:
                merged_rows.append(row.to_dict())

    return pd.DataFrame(merged_rows)




df_combined = merge_frames_list(arr_dfs, numeric_cols)
df_combined

In [ ]:
arr_dfs_going_right = []
# path_arr = paths[25:50]


for i, path in enumerate(GOING_RIGHT_PATHS[:6]):
    print(f"path = {path}")
    # path = "/Users/brianpennington/git/cfb/capture_stream/mnt/cfb_shared/screenshots/frame_20250802_171000_000049.jpg"
    data = run_easyocr_on_image(
        image_path = path, 
        use_gpu = False, 
        preprocess = False, 
        show_image = False,
        crop_loc = roster_data_loc
    )

    headers = [d[1] for d in data if d[0][0][1] <= 30]
    body_data = [d for d in data if d[0][0][1] > 30]
    grouped_rows = group_ocr_rows(body_data)
    df = build_table(headers, grouped_rows)
    arr_dfs_going_right.append(df)

In [ ]:
from scipy.spatial.distance import euclidean


def merge_columns_from_frame_going_right(master_df, new_df, max_distance=5.0):
    
    match_cols = master_df.columns.tolist()[-3:]

    def row_fingerprint(row):
        fp = []
        for col in match_cols:
            val = row.get(col)
            if pd.notna(val):
                try:
                    fp.append(float(val))
                except ValueError:
                    continue
        return tuple(fp)

    def similarity(fp1, fp2):
        if len(fp1) != len(fp2):
            return float('inf')
        return euclidean(fp1, fp2)

    # Add new columns to master_df if needed
    columns = []
    for col in new_df.columns:
        if col in CFB26_ATTRS:
            if col not in master_df.columns:
                master_df[col] = np.nan
            columns.append(col)
        else:
            print(f"col = {col} not in CFB26_ATTRS")

    new_df = new_df[columns]
    # Match each row in new_df to a row in master_df
    for _, new_row in new_df.iterrows():
        new_fp = row_fingerprint(new_row)

        best_match = None
        best_dist = float('inf')
        for idx, master_row in master_df.iterrows():
            master_fp = row_fingerprint(master_row)
            dist = similarity(new_fp, master_fp)
            if dist < best_dist and dist <= max_distance:
                best_dist = dist
                best_match = idx

        if best_match is not None:
            for col in new_df.columns:
                val = new_row[col]
                if pd.notna(val):
                    master_df.at[best_match, col] = val

    return master_df

# match_cols = ['THP', 'SAC', 'MAC']  # stable columns
merged_df = arr_dfs_going_right[0].copy(deep=True)
print(f"merged_df.columns = {merged_df.columns}")
for i in range(1, len(arr_dfs_going_right)):
    merged_df = merge_columns_from_frame_going_right(
        master_df=merged_df,
        new_df=arr_dfs_going_right[i],
        max_distance=5.0
    )

# merged_rows = merge_columns_from_frame(merged_rows, new_df, match_cols)
merged_df

In [ ]:
def merge_columns_from_frame_going_left(master_df, new_df, max_distance=5.0):
    # match_cols = master_df.columns.tolist()[:3]  # left-most stable cols

    def row_fingerprint(row):
        fp = []
        for col in match_cols:
            val = row.get(col)
            if pd.notna(val):
                try:
                    fp.append(float(val))
                except ValueError:
                    continue
        return tuple(fp)

    def similarity(fp1, fp2):
        if len(fp1) != len(fp2):
            return float('inf')
        return euclidean(fp1, fp2)

    # Filter to valid columns
    new_cols = []
    for col in new_df.columns:
        if col in CFB26_ATTRS:
            if col not in master_df.columns:
                master_df.insert(loc=0, column=col, value=np.nan)
            else:
                if (len(match_cols) < 3) and (col.upper() not in ['NAME', 'YEAR']):
                    match_cols.append(col)
            new_cols.append(col)
        else:
            print(f"col = {col} not in CFB26_ATTRS")

    new_df = new_df[new_cols]

    for _, new_row in new_df.iterrows():
        new_fp = row_fingerprint(new_row)
        best_idx = None
        best_dist = float('inf')
        for idx, master_row in master_df.iterrows():
            master_fp = row_fingerprint(master_row)
            dist = similarity(new_fp, master_fp)
            if dist < best_dist and dist <= max_distance:
                best_dist = dist
                best_idx = idx

        if best_idx is not None:
            for col in new_df.columns:
                val = new_row[col]
                if pd.notna(val):
                    master_df.at[best_idx, col] = val

    return master_df

# Scrolling left
arr_dfs_going_left = arr_dfs_going_right[::-1]
merged_df_going_left = arr_dfs_going_left[0].copy(deep=True)
for df in arr_dfs_going_left:
    merged_df_going_left = merge_columns_from_frame_going_left(
        master_df=merged_df_going_left,
        new_df=df,
        max_distance=5.0
    )

merged_df_going_left


In [ ]:
print(arr_dfs_going_left[0])

In [ ]:
print(arr_dfs_going_left[2])

In [ ]:
import numpy as np

def sad(a, b):
    return np.nansum(np.abs(a - b))

def detect_scroll_shift(prev_df, curr_df, numeric_cols, max_shift=5):

    shared_cols = [col for col in numeric_cols if col in prev_df.columns and col in curr_df.columns]
    prev = prev_df[shared_cols].astype(np.float32).values
    curr = curr_df[shared_cols].astype(np.float32).values

    best_score = float('inf')
    best_shift = (0, 0)

    for row_shift in range(-max_shift, max_shift + 1):
        for col_shift in range(-max_shift, max_shift + 1):
            # Define slice ranges
            r1s, r1e = max(0, row_shift), min(prev.shape[0], curr.shape[0] + row_shift)
            r2s, r2e = max(0, -row_shift), min(curr.shape[0], prev.shape[0] - row_shift)

            c1s, c1e = max(0, col_shift), min(prev.shape[1], curr.shape[1] + col_shift)
            c2s, c2e = max(0, -col_shift), min(curr.shape[1], prev.shape[1] - col_shift)

            if r1e - r1s <= 2 or c1e - c1s <= 2:
                continue  # skip if not enough overlap to compare

            prev_slice = prev[r1s:r1e, c1s:c1e]
            curr_slice = curr[r2s:r2e, c2s:c2e]

            score = sad(prev_slice, curr_slice)

            if score < best_score:
                best_score = score
                best_shift = (row_shift, col_shift)

    return best_shift, best_score


numeric_cols = ['OVR', 'SPD', 'ACC', 'AGI', 'COD', 'STR', 'AWR', 'THP', 'SAC', 'MAC', 'DAC', 'RUN', 'TUP', 'BSK']
shift, score = detect_scroll_shift(arr_dfs_going_left[0], arr_dfs_going_left[2], numeric_cols, max_shift=5)
print(f"Detected scroll: rows={shift[0]}, cols={shift[1]} | score={score}")

In [ ]:
def merge_rows_from_frame_going_up(master_df, new_df, match_cols, max_distance=5.0):
    from scipy.spatial.distance import euclidean

    def row_fingerprint(row):
        return tuple(float(row.get(c)) for c in match_cols if pd.notna(row.get(c)))

    def similarity(fp1, fp2):
        if len(fp1) != len(fp2):
            return float('inf')
        return euclidean(fp1, fp2)

    unmatched_new_rows = []

    for _, new_row in new_df.iterrows():
        new_fp = row_fingerprint(new_row)
        matched = False

        for _, master_row in master_df.iterrows():
            master_fp = row_fingerprint(master_row)
            dist = similarity(new_fp, master_fp)
            if dist <= max_distance:
                matched = True
                break

        if not matched:
            unmatched_new_rows.append(new_row)

    # Prepend unmatched rows
    if unmatched_new_rows:
        prepend_df = pd.DataFrame(unmatched_new_rows)
        master_df = pd.concat([prepend_df, master_df], ignore_index=True)

    return master_df

In [ ]:
def detect_horizontal_scroll(prev_df, curr_df):
    prev_cols = list(prev_df.columns)
    curr_cols = list(curr_df.columns)
    print(f"prev_cols = {prev_cols}")
    print(f"curr_cols = {curr_cols}")

    prev_right_col = prev_cols[-1]
    prev_left_col = prev_cols[0]
    if prev_cols == curr_cols:
        return None
    elif prev_right_col in curr_cols[:-1]:
        return "right"
    elif prev_left_col in curr_cols[1:]:
        return "left"
    else:
        return "horizontal_shift"  # unaligned but not single step



numeric_cols = ['OVR', 'SPD', 'ACC', 'AGI', 'COD', 'STR', 'AWR', 'THP', 'SAC', 'MAC', 'DAC', 'RUN', 'TUP', 'BSK']
# shift, score = detect_scroll_shift(arr_dfs_going_left[0], arr_dfs_going_left[2], numeric_cols, max_shift=5)
# print(f"Detected scroll: rows={shift[0]}, cols={shift[1]} | score={score}")

df_prev = arr_dfs_going_left[2]
df_curr = arr_dfs_going_left[0]
def detect_scroll(df_prev, df_curr):
    if horizontal := detect_horizontal_scroll(df_prev, df_curr):
        return horizontal
    else:
        row_shift, score = detect_scroll_shift(df_prev, df_curr, numeric_cols)
        if row_shift > 0:
            return "down"
        elif row_shift < 0:
            return "up"
        else:
            return "No scroll detected"

detect_scroll(df_prev, df_curr)